In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

In [8]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiGraph_60s_epoch = join(root_folder, 'ActiGraph_60s_epoch')
# activPAL_15s_epoch = join(root_folder, 'activPAL_15s_epoch')

# sleep time input_data
sleep_time_file = join(root_folder, 'Analytics_Scripts/Rashmika_Analysis/metadata/Copy of sleep_time.xlsx')

# Output
Actigraph_output_folder = join(root_folder, 'Analytics_Scripts/Rashmika_Analysis/processed_data/actigraph/epoch_data/60s_ref')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load Sleep Data

In [ ]:
sleep_df = pd.read_excel(sleep_time_file)

In [ ]:
sleep_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,av_inbedtime,av_outofbed_time
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00:00,06:00:00
1,1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d,161h 59m 59d,51.0,female,42.41690,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21:00:00,09:00:00
2,2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d,161h 59m 59d,56.0,male,30.09143,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,06:00:00
3,3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d,159h 59m 59d,61.0,female,27.43484,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00
4,4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d,NaT,NaT,NaN,NaN,56.0,female,36.73095,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00


### MET Equation

In [ ]:
def get_freedson_vm3_combination_11_energy_expenditure(row):

    if hip_epoch_data['waist_vm_cpm'][row.name] <= 2453:
        # METs = 0.001092(VA) + 1.336129  [capped at 2.9999, where VM3 < 2453]
        met_value = (0.001092 * hip_epoch_data['waist_cpm'][row.name]) + 1.336129
        met_value = met_value if met_value < 2.9999 else 2.9999
    else:
        # METs = 0.000863(VM3) + 0.668876 [where VM3 ≥ 2453]
        met_value = 0.000863 * hip_epoch_data['waist_vm_60'][row.name] + 0.668876

    return met_value

### Process ref values

In [ ]:
epoch_files = [f for f in listdir(ActiGraph_60s_epoch) if isfile(join(ActiGraph_60s_epoch, f))]
print(epoch_files)

['HOA1 (2019-05-06)60sec.csv', 'HOA2 (2019-05-06)60sec.csv', 'HOA3 (2019-05-06)60sec.csv', 'HOA4 (2019-05-06)60sec.csv', 'HOA6 (2019-05-20)60sec.csv', 'HOA7 (2019-05-27)60sec.csv', 'HOA8 (2019-05-27)60sec.csv', 'HOA9 (2019-05-27)60sec.csv', 'HOA10 (2019-06-24)60sec.csv', 'HOA11 (2019-07-01)60sec.csv', 'HOA12 (2019-07-01)60sec.csv', 'HOA14 (2019-07-15)60sec.csv', 'HOA15 (2019-07-29)60sec.csv', 'HOA16 (2019-07-29)60sec.csv', 'HOA17 (2019-07-29)60sec.csv', 'HOA18 (2019-07-29)60sec.csv', 'HOA19 (2019-08-05)60sec.csv', 'HOA20 (2019-08-19)60sec.csv', 'HOA22 (2019-08-19)60sec.csv', 'HOA25 (2019-09-02)60sec.csv', 'HOA26 (2019-09-02)60sec.csv', 'HOA27 (2019-09-02)60sec.csv', 'HOA29 (2019-09-02)60sec.csv', 'HOA30 (2019-09-02)60sec.csv', 'HOA33 (2019-09-09)60sec.csv', 'HOA34 (2019-09-23)60sec.csv', 'HOA35 (2019-09-23)60sec.csv', 'HOA36 (2019-09-30)60sec.csv', 'HOA37 (2019-09-30)60sec.csv', 'HOA39 (2019-10-07)60sec.csv', 'HOA40 (2019-10-07)60sec.csv', 'HOA41 (2019-10-07)60sec.csv', 'HOA42 (2019-10

In [ ]:
epoch_files[156]

'OA46_fixed60sec.csv'

In [ ]:
#check#OA46
# oafile = join(ActiGraph_60s_epoch, epoch_files[156])
# print(oafile)
# oa46 = pd.read_csv(oafile, skiprows=11)
# oa46.head()

sleep_df.loc[sleep_df['Participant'] == 'OA46']

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,av_inbedtime,av_outofbed_time
500,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,1.0,0.0,333.0,0.0,NaN,1.0,2016-03-07 21:45:00,2016-03-08 03:17:00
501,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,2.0,0.0,212.0,0.0,NaN,1.0,2016-03-09 01:39:00,2016-03-09 05:10:00
502,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,3.0,0.0,188.0,0.0,NaN,1.0,2016-03-09 21:58:00,2016-03-10 01:05:00
503,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,4.0,0.0,377.0,0.0,NaN,1.0,2016-03-12 01:03:00,2016-03-12 07:19:00
504,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,5.0,0.0,486.0,0.0,NaN,1.0,2016-03-12 23:00:00,2016-03-13 07:05:00


In [ ]:
for f in tqdm(epoch_files):

  user_id = f.split()[0]

  hip_epoch_data = pd.read_csv(join(ActiGraph_60s_epoch, f), skiprows=11, usecols=[0, 1, 2], header=None)
  # Axis 1 (y) - Goes through head and middle of feet
  # Axis 2 (x) - Goes through 2 hips
  # Axis 3 (z) - Goes through front and back of the stomach
  hip_epoch_data.columns = ['AxisY', 'AxisX', 'AxisZ']

  # Create time column
  print(user_id)
  if user_id != 'OA46_fixed60sec.csv':
    start_time = sleep_df.loc[sleep_df['Participant'] == user_id].iloc[0]['actigraph_Start_time']
  else:
    start_time = "07-03-2016  4:00:00"

  hip_epoch_data['Time_stamp'] = list(pd.date_range(start_time, periods=hip_epoch_data.shape[0], freq='min'))
  # print(hip_epoch_data['Time_stamp'])
  # break
  hip_epoch_data['waist_vm_60'] = np.sqrt([(hip_epoch_data.AxisX ** 2) + (hip_epoch_data.AxisY ** 2) + (hip_epoch_data.AxisZ ** 2)])[0]
  hip_epoch_data['waist_vm_cpm'] = hip_epoch_data['waist_vm_60']  # cpm = counts per minute
  hip_epoch_data['waist_cpm'] = hip_epoch_data.AxisY

  hip_epoch_data['waist_ee'] = hip_epoch_data.apply(get_freedson_vm3_combination_11_energy_expenditure, axis=1)

  # # Use following snippet if need the reference data for granular time_epochs
  # multiplication_factor = int(60 / time_epoch)
  # epoch_hip_epoch_data = epoch_hip_epoch_data.reindex(np.repeat(epoch_hip_epoch_data.index.values, multiplication_factor), method='ffill')

  # save output file
  # Input example: 'HOA1 (2019-05-06)60sec.csv'
  output_filename = f.replace('.csv', '_60sREF.csv')
  hip_epoch_data.to_csv(join(Actigraph_output_folder, output_filename), sep=',', index=None)
  del hip_epoch_data


  0%|          | 0/217 [00:00<?, ?it/s]

HOA1


  0%|          | 1/217 [00:06<22:18,  6.20s/it]

HOA2


  1%|          | 2/217 [00:07<17:19,  4.84s/it]

...

 99%|█████████▉| 215/217 [06:19<00:02,  1.08s/it]

HOA49


100%|█████████▉| 216/217 [06:20<00:01,  1.06s/it]

HOA53


100%|██████████| 217/217 [06:22<00:00,  1.76s/it]
